# The Communication Value of Noisy Quantum Channels

The communication value quantifies the performance of classical communication over a black-box (quantum) channel.
For a quantum channel $\mathcal{N}$, the communication value is defined as

$$
\text{cv}(\mathcal{N}) = \max_{\{\rho_x\}_x,\{\Pi_y\}_y} \sum_{x=y} \text{Tr}[\Pi_y \mathcal{N}(\rho_x)].
$$

If the Hilbert spaces of $\rho_x$ and $\Pi_y$ are both dimension $d$, then it is sufficient to consider $x,y\in[d^2]:=\{1,2,\dots,d^2\}$.
General states and POVMs should be considered, however, for simplicity, we'll consider pure states and combinations of PVM measurements.

In [1]:
import pennylane as qml
import pennylane.numpy as np
import matplotlib.pyplot as plt

We'll begin with ideal, single-qubit communication with 4 state encodings and 4 measurement outcomes.

In [55]:
pure_qubit_dev = qml.device("default.qubit", wires=1)

@qml.qnode(pure_qubit_dev)
def cv_qubit_circuit(settings):
    (θ_state, θ_measurement) = settings
    qml.RY(θ_state, wires=0)
    qml.RY(θ_measurement, wires=0)
    
    # return array probs
    # probs[0] -> +1 and probs[1] -> -1
    return qml.probs(wires=0)

In [21]:
cv_qubit_circuit((np.pi,0))

tensor([3.74939946e-33, 1.00000000e+00], requires_grad=True)

In [22]:
print(cv_qubit_circuit.draw())

 0: ──RY(3.14)──RY(0)──┤ Probs 



In [65]:
noisy_qubit_dev = qml.device('default.mixed', wires=1)

@qml.qnode(noisy_qubit_dev)
def cv_noisy_qubit_circuit(settings):
    (θ_state, θ_measurement) = settings
    qml.RY(θ_state, wires=0)
    qml.AmplitudeDamping(0.5, wires=0)
    qml.RY(θ_measurement, wires=0)
    
    # return array probs
    # probs[0] -> +1 and probs[1] -> -1
    return qml.probs(wires=0)

In [66]:
def cv_settings(d):
    θ_states = 2*np.pi*np.random.random(d**2) - np.pi
    θ_measurements = 2*np.pi*np.random.random(d**2//2) - np.pi
    
    return (θ_states, θ_measurements)

def cv_cost(settings):
    θ_states, θ_measurements = settings
    
    d2 = len(θ_states)
    
    cv = 0
    
    for x in range(d2):
        y = x//2
        
        probs = cv_qubit_circuit((θ_states[x], θ_measurements[y]))
        
        if x % 2 == 0:
            cv += probs[0]
        else:
            cv += probs[1]
    
    return -1/np.sqrt(d2)*(cv)


def cv_noisy_cost(settings):
    θ_states, θ_measurements = settings
    
    d2 = len(θ_states)
    
    cv = 0
    
    for x in range(d2):
        y = x//2
        
        probs = cv_noisy_qubit_circuit((θ_states[x], θ_measurements[y]))
        
        if x % 2 == 0:
            cv += probs[0]
        else:
            cv += probs[1]
    
    return -1/np.sqrt(d2)*(cv)

The BB84 states can be used to maximize the communication value.

In [47]:
θ_bb84_states = [0, np.pi, np.pi/2, 3*np.pi/2]
θ_bb84_measurements = [0, np.pi/2]

cv_cost((θ_bb84_states, θ_bb84_measurements))

tensor(-0.5, requires_grad=True)

In [67]:
opt = qml.GradientDescentOptimizer()    
d = 2
num_steps = 1000

# initial settings
settings = cv_settings(d)
cvs = []

# performing gradient descent
for i in range(num_steps):
    settings = opt.step(cv_noisy_cost, settings)

    if i%50 == 0:
        cv = -(cv_noisy_cost(settings))
        cvs.append(cv)

        print("iteration : ",i, ", cv : ", cv)
        print("settings :\n", settings, "\n")


final_cv = -(cv_noisy_cost(settings))
final_settings = settings

print("final cv : ", final_cv)
print("final settings : ", settings, "\n")

iteration :  0 , cv :  0.8575955153891022
settings :
 [array([ 2.72386409, -1.76600504, -0.02523962, -1.67253383]), array([-0.64588377,  2.12088429])] 

iteration :  50 , cv :  0.9144224506755869
settings :
 [array([ 2.65158301, -1.800314  , -0.10278249, -1.64713797]), array([-0.76876138,  2.1053115 ])] 

iteration :  100 , cv :  0.9699331444429267
settings :
 [array([ 2.57420457, -1.82575518, -0.18287886, -1.62069717]), array([-0.88554793,  2.0863672 ])] 

iteration :  150 , cv :  1.0225647344467572
settings :
 [array([ 2.49466299, -1.84274108, -0.26497092, -1.59330495]), array([-0.99380831,  2.06433423])] 

iteration :  200 , cv :  1.0711125342973
settings :
 [array([ 2.41572914, -1.85213686, -0.34834806, -1.56503624]), array([-1.09175112,  2.03968323])] 

iteration :  250 , cv :  1.1148718039503018
settings :
 [array([ 2.33967977, -1.85506243, -0.43217339, -1.53593673]), array([-1.17836719,  2.01306252])] 

iteration :  300 , cv :  1.1536263187380917
settings :
 [array([ 2.26812974,

In [62]:
cv_cost(final_settings)

-1.983537915328439

In [43]:
circuit((final_settings[0][0],final_settings[1][0])) 

tensor([9.99162603e-01, 8.37397312e-04], requires_grad=True)

In [44]:
circuit((final_settings[0][1],final_settings[1][0])) 

tensor([6.62981750e-04, 9.99337018e-01], requires_grad=True)

In [70]:
from pennylane.operation import Channel

class TwoQubitEntangledPauli(Channel):

    num_params = 1
    num_wires = 2
    par_domain = "R"
    grad_method = "A"
    grad_recipe = ([[1, 0, 1], [-1, 0, 0]],)

    @classmethod
    def _kraus_matrices(cls, *params):
        p = params[0]
        σx = np.array([[0, 1], [1, 0]])
        σy = np.array([[0, -1j], [1j, 0]])
        σz = np.array([[1, 0], [0, -1]])
        
        K0 = np.sqrt(1 - p) * np.eye(4)
        K1 = np.sqrt(p / 3) * np.kron(σx,σx)
        K2 = np.sqrt(p / 3) * np.kron(σy,σy)
        K3 = np.sqrt(p / 3) * np.kron(σz,σz)
        return [K0, K1, K2, K3]

In [130]:
noisy_2qubit_dev = qml.device('default.mixed', wires=2)

# making mixed simulator aware of the new channel
noisy_2qubit_dev.operations.update(["TwoQubitEntangledPauli"])

@qml.qnode(noisy_2qubit_dev)
def cv_noisy_2qubit_circuit(settings):
    (θ_state, θ_measurement) = settings
    
    p=1
    
    qml.RY(θ_state[0], wires=0)
    qml.RY(θ_state[1], wires=1)
    qml.CNOT(wires=[0,1])
    
    TwoQubitEntangledPauli(p, wires=[0,1])
    
    qml.CNOT(wires=[0,1])
    qml.RY(θ_measurement[0], wires=0)
    qml.RY(θ_measurement[1], wires=1)
    
    # return array probs
    # probs[0] -> +1 and probs[1] -> -1
    return qml.probs(wires=[0,1])

In [105]:
cv_noisy_2qubit_circuit((0.75,[0,0],[0,0]))

tensor([0.5, 0. , 0.5, 0. ], requires_grad=True)

In [135]:
def cv_2qubit_settings(d):
    θ_states = 2*np.pi*np.random.random((d**2,2)) - np.pi
    θ_measurements = 2*np.pi*np.random.random((d**2//4,2)) - np.pi
    
    return (θ_states, θ_measurements)

def cv_noisy_2qubit_cost(settings):
    θ_states, θ_measurements = settings
    
    d2 = len(θ_states)
    
    cv = 0
    
    for x in range(d2):
        y = x//4
        
        probs = cv_noisy_2qubit_circuit((θ_states[x], θ_measurements[y]))
        
        cv += probs[x % 4]
    
    return -1/np.sqrt(d2)*(cv)

In [132]:
opt = qml.GradientDescentOptimizer(stepsize=0.1)    
d = 4
num_steps = 500

# initial settings
settings = cv_2qubit_settings(d)
cvs = []

# performing gradient descent
for i in range(num_steps):
    settings = opt.step(cv_noisy_2qubit_cost, settings)

    if i%50 == 0:
        cv = -(cv_noisy_2qubit_cost(settings))
        cvs.append(cv)

        print("iteration : ",i, ", cv : ", cv)
        print("settings :\n", settings, "\n")


final_cv = -(cv_noisy_2qubit_cost(settings))
final_settings = settings

print("final cv : ", final_cv)
print("final settings : ", settings, "\n")

iteration :  0 , cv :  1.0359137336644157
settings :
 [array([[ 0.42130235,  2.88843209],
       [ 1.61824802, -2.07866868],
       [ 0.15967725,  0.6149277 ],
       [ 2.03157369,  1.49974496],
       [-2.08242323,  0.79554017],
       [ 2.00637766,  0.94109437],
       [ 1.88577926, -0.81340482],
       [ 0.60533786, -0.91582426],
       [-0.33153995, -1.17417465],
       [-2.72962982, -1.6980682 ],
       [-1.25011532,  0.64342075],
       [ 1.28968659, -0.36151258],
       [-2.208989  ,  0.39360606],
       [ 1.45699185, -2.17337891],
       [-1.94327078,  0.82458999],
       [ 3.0740473 , -1.4942581 ]]), array([[ 2.00355419,  1.56602853],
       [ 1.6553518 , -1.4017172 ],
       [ 1.68672554, -2.43537507],
       [-2.27838082, -2.69813524]])] 

iteration :  50 , cv :  1.4320867292533421
settings :
 [array([[ 0.13048902,  2.86184269],
       [ 1.63012026, -2.0696825 ],
       [ 0.47116039,  0.75736866],
       [ 2.04675021,  1.48367988],
       [-1.93552295,  0.51033828],
       [

final cv :  3.2574623781252776
final settings :  [array([[-1.53443236e+00,  3.11501865e+00],
       [ 1.93096221e+00, -1.89757461e+00],
       [ 1.55822993e+00,  3.05810592e+00],
       [ 1.60073691e+00,  1.20481357e-02],
       [-1.59280695e+00, -1.00576910e-02],
       [ 4.42062361e+00,  2.87692153e+00],
       [ 1.55111798e+00, -1.54221744e-02],
       [ 1.54169850e+00, -3.13429993e+00],
       [-1.56908424e+00, -3.13399693e+00],
       [-1.57932002e+00, -7.35966201e-03],
       [-8.83311118e-01,  8.79445773e-01],
       [ 1.56640031e+00, -1.35114268e-03],
       [-2.56249937e+00,  8.41616981e-01],
       [ 1.56686834e+00, -2.43218202e-02],
       [-1.57635546e+00,  3.11515384e+00],
       [ 4.70125660e+00, -2.13745891e-02]]), array([[ 1.56855551,  3.12362131],
       [ 1.53377646, -0.02826761],
       [ 1.55765202, -3.14987385],
       [-1.56181423, -3.16982976]])] 



In [141]:
noisy_2qubit_prod_dev = qml.device('default.mixed', wires=4)

# making mixed simulator aware of the new channel
noisy_2qubit_prod_dev.operations.update(["TwoQubitEntangledPauli"])

@qml.qnode(noisy_2qubit_prod_dev)
def cv_noisy_2qubit_prod_circuit(settings):
    (θ_state, θ_measurement) = settings
    
    p=1
    
    qml.RY(θ_state[0], wires=0)
    qml.RY(θ_state[1], wires=1)
    qml.RY(θ_state[2], wires=2)
    qml.RY(θ_state[3], wires=3)
    
    qml.CNOT(wires=[0,1])
    qml.CNOT(wires=[1,2])
    qml.CNOT(wires=[2,3])
    qml.CNOT(wires=[3,0])
    
    TwoQubitEntangledPauli(p, wires=[0,1])
    TwoQubitEntangledPauli(p, wires=[2,3])
    
    qml.CNOT(wires=[3,0])
    qml.CNOT(wires=[2,3])
    qml.CNOT(wires=[1,2])
    qml.CNOT(wires=[0,1])
    
    qml.RY(θ_measurement[0], wires=0)
    qml.RY(θ_measurement[1], wires=1)
    qml.RY(θ_measurement[2], wires=2)
    qml.RY(θ_measurement[3], wires=3)
    
    # return array probs
    # probs[0] -> +1 and probs[1] -> -1
    return qml.probs(wires=[0,1,2,3])

def cv_2qubit_prod_settings(d):
    θ_states = 2*np.pi*np.random.random((d**2,4)) - np.pi
    θ_measurements = 2*np.pi*np.random.random((d**2//16,4)) - np.pi
    
    return (θ_states, θ_measurements)

def cv_noisy_2qubit_prod_cost(settings):
    θ_states, θ_measurements = settings
    
    d2 = len(θ_states)
    
    cv = 0
    
    for x in range(d2):
        y = x//16
        
        probs = cv_noisy_2qubit_prod_circuit((θ_states[x], θ_measurements[y]))
                
        cv += probs[x % 16]
    
    return -(cv)/np.sqrt(d2)

In [ ]:
opt = qml.GradientDescentOptimizer(stepsize=0.1)    
d = 16
num_steps = 500

# initial settings
settings = cv_2qubit_prod_settings(d)
cvs = []

# performing gradient descent
for i in range(num_steps):
    settings = opt.step(cv_noisy_2qubit_prod_cost, settings)
    
    if i%5 == 0:
        print("iteration : ", i)
        
    if i%50 == 0:
        cv = -(cv_noisy_2qubit_prod_cost(settings))
        cvs.append(cv)
        
        print("cv : ", cv)
        print("settings :\n", settings, "\n")

final_cv = -(cv_noisy_2qubit_prod_cost(settings))
final_settings = settings

print("final cv : ", final_cv)
print("final settings : ", settings, "\n")

iteration :  0
cv :  0.8922580049344598
settings :
 [array([[ 2.95269287,  0.48590527, -2.27445125, -2.47765382],
       [-3.02402407,  0.07686326, -2.31076757,  1.05406172],
       [-1.34694423, -3.12739108,  1.93993103, -1.49031001],
       ...,
       [-0.29618177,  1.10869241, -0.45340009, -1.58485345],
       [ 0.51266991, -1.20111158,  1.61534229,  0.39731458],
       [-2.32115559,  0.95353403,  0.15561556,  2.00861528]]), array([[-2.28625551,  1.50829223,  0.40408182,  1.02842784],
       [-0.27103285, -1.81975777,  1.59325933,  1.937261  ],
       [-2.54527055,  0.89350472,  2.45216282, -1.85824131],
       [ 0.36169689,  0.17491945,  0.55472951,  2.85303455],
       [ 2.35934657, -0.62469922,  0.85738973, -0.24524303],
       [-2.36453428,  1.79723052,  0.72189803,  1.90999546],
       [ 2.42402396,  0.24405383, -1.54052127, -0.832667  ],
       [ 2.70189088, -0.09937289, -0.84969576,  0.99029213],
       [ 1.32477178,  2.28969668, -0.11093777, -2.64727251],
       [ 1.1120997